# This Notebook is code to accompany article Automatic Notification to Email with Python

Article: https://towardsdatascience.com/automatic-notification-to-email-with-python-810fd357d89c?sk=7f8636d27b728b32d22de011560eae6d

In [1]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import base64
import os

def create_message(sender, to, subject, message_text):
  """Create a message for an email.

  Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

  Returns:
    An object containing a base64url encoded email object.
  """
  message = MIMEText(message_text)
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  return {'raw': base64.urlsafe_b64encode(message.as_string().encode()).decode()}



def send_message(service, user_id, message):
  """Send an email message.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

  Returns:
    Sent Message.
  """
  try:
    message = (service.users().messages().send(userId=user_id, body=message)
               .execute())
    print('Message Id: %s' % message['id'])
    return message
  except:
    print ('An error occurred')

In [4]:
def notification(sender, to, subject, notification):

    SCOPES = 'https://mail.google.com/'
    message = create_message(sender, to, subject, notification)
    creds = None

    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    send_message(service, sender, message)

In [17]:
from sklearn.linear_model import LinearRegression
mpg = sns.load_dataset('mpg').drop(['origin', 'name'], axis = 1)
linear_model = LinearRegression()

try:
    linear_model.fit(mpg.drop('mpg', axis =1), mpg['mpg'])
    notification('cornelliusyudhaw@gmail.com', 'cornelliusyudhaw@gmail.com', 'Notification - Success Training', 'The model has finish')
except:
    notification('cornelliusyudhaw@gmail.com', 'cornelliusyudhaw@gmail.com', 'Notification - Failed Training', 'The model encountered error')
    

Message Id: 172ae298cd3c8ec8
